In [0]:
#creating SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('TDSQL').getOrCreate()

In [0]:
#creating dataframe
df = spark.read.csv('/FileStore/tables/taxi_data.csv',inferSchema=True,header=True)

In [0]:
data=df.dropna()
data.count()

Out[3]: 995134

In [0]:
#Converted Date columns from StringType to timestamp 
from pyspark.sql.functions import *

spark.conf.set("spark.sql.legacy.timeParserPolicy", "Legacy")
data=data.withColumn('tpep_pickup_datetime',to_timestamp(data.tpep_pickup_datetime, 'MM/dd/yyyy HH:mm')).\
        withColumn('tpep_dropoff_datetime',to_timestamp(data.tpep_dropoff_datetime, 'MM/dd/yyyy HH:mm'))

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.functions import *

# Engineered features
# Get rid of 27 categories into a dummy for condition. bad conditions include any of the categories below
# Temperature should only make a difference when causes an inconvenience/discomfort
# Get date, year, month, hour, day from tpep_pickup to perform analysis 
# Get trip duration and Covid

data = data.withColumn("good_condition", when(f.col("condition") == "Snow", 0) \
                        .when(f.col("condition") == "Rain / Windy", 0) \
                        .when(f.col("condition") == "Heavy Rain", 0) \
                        .when(f.col("condition") == "Rain", 0) \
                        .when(f.col("condition") == "Heavy T-Storm", 0) \
                        .when(f.col("condition") == "Thunder in the Vicinity", 0) \
                        .when(f.col("condition") == "Thunder", 0) \
                        .when(f.col("condition") == "Light Rain with Thunder", 0) \
                        .when(f.col("condition") == "Thunder / Windy", 0) \
                        .when(f.col("condition") == "T-Storm", 0) \
                        .otherwise(1)) \
            .withColumn("extreme_temp", when((f.col("temperature") > 86) | (f.col("temperature") < 21), 1).otherwise(0)) \
            .withColumn('date',to_date(data.tpep_pickup_datetime)) \
            .withColumn('year',year(data.tpep_pickup_datetime)) \
            .withColumn('month',month(data.tpep_pickup_datetime)) \
            .withColumn('hour', hour(data.tpep_pickup_datetime)) \
            .withColumn('day', dayofweek(data.tpep_pickup_datetime)) \
            .withColumn('trip_time', unix_timestamp(data.tpep_dropoff_datetime) - unix_timestamp(data.tpep_pickup_datetime)) \
            .withColumn('covid', when(f.col("tpep_pickup_datetime") > "2020-03-08 00:00:00", 1).otherwise(0)) 

data.show(1)            

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+----------+--------------+----------+---------+-------+------------+--------------+----------+--------------+----------+---------+-------+------------+--------------+-----------+-----------+--------+----------+--------+------+----------+--------------+------------+----------+----+-----+----+---+---------+-----+
|vendorid|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|store_and_fwd_flag|pulocationid|dolocationid|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|borough_pu|median_rlst_pu|tourist_pu|entert_pu|park_pu|workplace_pu|residential_pu|borough_do|median_rlst_do|tourist_do|entert_do|park_do|workplace_do|residential_do|  rat

In [0]:
# creating view from dataframe
data.createOrReplaceTempView("TD")

In [0]:
# Running sql

spark.sql("describe TD").show(50)

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|            vendorid|      int|   null|
|tpep_pickup_datetime|timestamp|   null|
|tpep_dropoff_date...|timestamp|   null|
|     passenger_count|      int|   null|
|       trip_distance|   double|   null|
|          ratecodeid|      int|   null|
|  store_and_fwd_flag|   string|   null|
|        pulocationid|      int|   null|
|        dolocationid|      int|   null|
|        payment_type|      int|   null|
|         fare_amount|   double|   null|
|               extra|   double|   null|
|             mta_tax|   double|   null|
|          tip_amount|   double|   null|
|        tolls_amount|   double|   null|
|improvement_surch...|   double|   null|
|        total_amount|   double|   null|
|congestion_surcharge|   double|   null|
|          borough_pu|   string|   null|
|      median_rlst_pu|      int|   null|
|          tourist_pu|      int|   null|
|           ente

#### 1.Yearly Tip and Fare variation analysis

In [0]:
dy=spark.sql("select year, count(*) from TD group by year ")
dy.show()
display(dy)

+----+--------+
|year|count(1)|
+----+--------+
|2019|  695707|
|2020|  299427|
+----+--------+



year,count(1)
2019,695707
2020,299427


In [0]:
yft=spark.sql("select year, avg(fare_amount) avg_fare, avg(tip_amount) avg_tip from TD group by year ")
yft.show()
display(yft)

+----+------------------+------------------+
|year|          avg_fare|           avg_tip|
+----+------------------+------------------+
|2019|13.520902951961098|3.1059295220544243|
|2020|12.415011338322863|2.9158831701884704|
+----+------------------+------------------+



year,avg_fare,avg_tip
2019,13.520902951961098,3.1059295220544243
2020,12.415011338322863,2.9158831701884704


#### 2.Monthly Tip and Fare variation analysis:

In [0]:
mtf = spark.sql("Select month, avg(tip_amount) avg_tip,  avg(fare_amount) avg_fare from TD group by month order by month")
mtf.show()
display(mtf)

+-----+------------------+------------------+
|month|           avg_tip|          avg_fare|
+-----+------------------+------------------+
|    1|2.9498423074917173|12.507652693986126|
|    2| 2.935678410600622|12.508745233699685|
|    3|2.8672593022385233|12.303630119038706|
|    4| 2.592460667086219|10.593030207677785|
|    5| 2.603315048891009|11.096243739565944|
|    6|2.7507827559260223| 11.86077103412347|
|    7| 3.085359244433802|13.573856966298353|
|    8|3.0866147852399113|13.521797739774362|
|    9|3.1231177613680776|13.747278144837415|
|   10|3.1064596668420967|13.478330833208979|
|   11|3.0711107997395786| 13.28658443709974|
|   12|  3.15951500381065|13.536070031741247|
+-----+------------------+------------------+



month,avg_tip,avg_fare
1,2.9498423074917173,12.507652693986126
2,2.935678410600622,12.508745233699685
3,2.8672593022385233,12.303630119038706
4,2.592460667086219,10.593030207677785
5,2.603315048891009,11.096243739565944
6,2.7507827559260223,11.86077103412347
7,3.085359244433802,13.573856966298353
8,3.0866147852399113,13.521797739774362
9,3.1231177613680776,13.747278144837415
10,3.1064596668420967,13.478330833208979


##### Fall/Summer significantly higher than Winter/Spring

####3. Day of the week Vs Trips, Fare and number of trips

In [0]:
#1 = Sunday, 2 = Monday, ..., 7 = Saturday
dtf = spark.sql("Select day, avg(fare_amount) as avg_fare, avg(tip_amount) as avg_tip,count(*) number_of_trips from TD group by day order by day")
dtf.show()
display(dtf)

+---+------------------+------------------+---------------+
|day|          avg_fare|           avg_tip|number_of_trips|
+---+------------------+------------------+---------------+
|  1|13.524821936404019| 3.058536398862472|         116391|
|  2|13.408537604243316| 3.097147948859482|         130464|
|  3|12.932413096608698| 3.023846842661639|         148527|
|  4|13.125009528657763|3.0717772904674607|         153222|
|  5|13.380171119821748|3.1216165173865926|         158427|
|  6|13.357219004760974|3.0736167090589332|         153330|
|  7|12.619604371795537|2.8807393914211605|         134773|
+---+------------------+------------------+---------------+



day,avg_fare,avg_tip,number_of_trips
1,13.524821936404019,3.058536398862472,116391
2,13.408537604243316,3.097147948859482,130464
3,12.932413096608698,3.023846842661639,148527
4,13.125009528657763,3.0717772904674607,153222
5,13.380171119821748,3.1216165173865926,158427
6,13.357219004760974,3.0736167090589332,153330
7,12.619604371795537,2.8807393914211605,134773


#### 4.Time of day Fare and Tip analysis

In [0]:
hft = spark.sql("Select hour, avg(tip_amount) avg_tip,avg(fare_amount) avg_fare from TD group by hour order by hour")
hft.show()
display(hft)

+----+------------------+------------------+
|hour|           avg_tip|          avg_fare|
+----+------------------+------------------+
|   0| 3.154819584079322| 13.90437543038149|
|   1|2.8937232036522444|12.743796149265581|
|   2| 2.771901818181822|12.127593454545455|
|   3|2.7623483842408163|12.617586321381143|
|   4|3.4887446878064745| 16.35285550833606|
|   5|3.9108726845812716|18.006258805113486|
|   6| 3.054725844542559|13.862902248690833|
|   7|2.7756223809398275|12.313492343778826|
|   8|2.8074344023323534|12.374243009074858|
|   9| 2.865102123142241|12.550734607218686|
|  10|2.9298888401666225|12.615424687568513|
|  11|2.9919351488820563|12.892278054466463|
|  12|3.0046160247265763| 13.02047075606277|
|  13| 3.107032357109113| 13.45909443394741|
|  14| 3.173931352592953|13.915869764141254|
|  15|3.1635382084365387|13.899432696217975|
|  16| 3.302205168847741|14.079680861345754|
|  17|3.1592528147390024|13.437773302076797|
|  18|3.0018690093501856|12.663791888678496|
|  19|2.97

hour,avg_tip,avg_fare
0,3.154819584079322,13.90437543038149
1,2.8937232036522444,12.743796149265581
2,2.771901818181822,12.127593454545455
3,2.7623483842408163,12.617586321381143
4,3.4887446878064745,16.35285550833606
5,3.9108726845812716,18.006258805113486
6,3.054725844542559,13.862902248690833
7,2.7756223809398275,12.313492343778826
8,2.8074344023323534,12.374243009074858
9,2.865102123142241,12.550734607218686


In [0]:
hf = spark.sql("Select hour,avg(fare_amount) avg_fare, count(*) number_of_trips from TD group by hour order by avg_fare desc")
hf.show()

+----+------------------+---------------+
|hour|          avg_fare|number_of_trips|
+----+------------------+---------------+
|   5|18.006258805113486|           7666|
|   4| 16.35285550833606|           6118|
|  16|14.079680861345754|          51733|
|  14|13.915869764141254|          53549|
|   0| 13.90437543038149|          29044|
|  15|13.899432696217975|          54098|
|   6|13.862902248690833|          19478|
|  23|13.692730290660505|          40666|
|  13| 13.45909443394741|          50808|
|  17|13.437773302076797|          60574|
|  22|13.401472365014447|          53302|
|  21|13.195735154127073|          57777|
|  12| 13.02047075606277|          50472|
|  20|12.997761527316117|          58665|
|  11|12.892278054466463|          47185|
|   1|12.743796149265581|          20152|
|  18|12.663791888678496|          68127|
|   3|12.617586321381143|           9036|
|  10|12.615424687568513|          45610|
|   9|12.550734607218686|          47100|
+----+------------------+---------

##### High Tip hours : 2pm - 6pm, 9pm - 12am. People tend to give more tips during these hours
##### Highest fare trips are around the end of business hours between 2pm - 5pm

#### 5.COVID Tip and Fare analysis:

In [0]:
cd =spark.sql("select covid, count(*) from TD group by covid ")
cd.show()
display(cd)

+-----+--------+
|covid|count(1)|
+-----+--------+
|    1|   40696|
|    0|  954438|
+-----+--------+



covid,count(1)
1,40696
0,954438


In [0]:
cft = spark.sql("Select covid, avg(fare_amount) avg_fare,avg(tip_amount) avg_tip from TD group by covid order by avg_fare desc")
cft.show()
display(cft)

+-----+------------------+-----------------+
|covid|          avg_fare|          avg_tip|
+-----+------------------+-----------------+
|    0|13.246076612624394|3.060755177392359|
|    1|11.829603892274426|2.767102663652443|
+-----+------------------+-----------------+



covid,avg_fare,avg_tip
0,13.246076612624394,3.060755177392359
1,11.829603892274426,2.767102663652443


##### tips and fares for taxi rides that occurred during New York’s COVID shutdown were lower on average than trips that happened prior to the shutdowns.

#### 6. Weather Tip and Fare effect analysis

In [0]:
wft = spark.sql("Select good_condition, avg(tip_amount) avg_tip, avg(fare_amount) avg_fare,count(*) from TD group by good_condition order by avg_fare desc")
wft.show()
display(wft)

+--------------+------------------+------------------+--------+
|good_condition|           avg_tip|          avg_fare|count(1)|
+--------------+------------------+------------------+--------+
|             0|3.0912071113666864| 13.23720816561242|   20868|
|             1| 3.047836781741349|13.187099180305996|  974266|
+--------------+------------------+------------------+--------+



good_condition,avg_tip,avg_fare,count(1)
0,3.0912071113666864,13.23720816561242,20868
1,3.047836781741349,13.187099180305996,974266


##### Poor weather conditions( such as thunderstorms, heavy snow or rain) or favorable weather conditions (such as fair or overcast) did not have a significant effect on tips or fare

In [0]:
ttf = spark.sql("Select extreme_temp, avg(tip_amount) avg_tip, avg(fare_amount) avg_fare, count(*) from TD group by extreme_temp order by avg_fare desc")
ttf.show()
display(ttf)

+------------+------------------+------------------+--------+
|extreme_temp|           avg_tip|          avg_fare|count(1)|
+------------+------------------+------------------+--------+
|           1|3.1230487663347453| 13.64918923937516|   43083|
|           0|3.0453838607385646|13.167286636955374|  952051|
+------------+------------------+------------------+--------+



extreme_temp,avg_tip,avg_fare,count(1)
1,3.1230487663347453,13.64918923937516,43083
0,3.0453838607385646,13.167286636955374,952051


##### Extreme temperatures did not have a significant effect on tips or fare

#### 7. Tip and Fare effect analysis based on location

In [0]:
lft = spark.sql("Select borough_pu, avg(tip_amount) avg_tip, avg(fare_amount) avg_fare,count(*) number_of_trips from TD where borough_pu != 'Unknown' group by borough_pu  ")
lft.show()
display(lft)

+-------------+------------------+------------------+---------------+
|   borough_pu|           avg_tip|          avg_fare|number_of_trips|
+-------------+------------------+------------------+---------------+
|       Queens| 8.171751803301419| 39.75105319739215|          57672|
|          EWR| 4.840952380952381| 61.63333333333333|             21|
|     Brooklyn| 2.903390184476158|16.712644158255017|           8619|
|Staten Island|1.2273684210526317| 66.76315789473684|             38|
|    Manhattan| 2.727202888749636|11.456650322558236|         921229|
|        Bronx|1.4230975954738332|23.388854314002828|            707|
+-------------+------------------+------------------+---------------+



borough_pu,avg_tip,avg_fare,number_of_trips
Queens,8.171751803301419,39.75105319739215,57672
EWR,4.840952380952381,61.63333333333333,21
Brooklyn,2.903390184476158,16.712644158255017,8619
Staten Island,1.2273684210526317,66.76315789473684,38
Manhattan,2.727202888749636,11.456650322558236,921229
Bronx,1.4230975954738332,23.388854314002828,707


In [0]:
lf = spark.sql("Select borough_pu, avg(fare_amount) avg_fare,count(*) number_of_trips from TD group by borough_pu order by avg_fare desc")
lf.show()


+-------------+------------------+---------------+
|   borough_pu|          avg_fare|number_of_trips|
+-------------+------------------+---------------+
|Staten Island| 66.76315789473684|             38|
|          EWR| 61.63333333333333|             21|
|       Queens| 39.75105319739215|          57672|
|        Bronx|23.388854314002828|            707|
|     Brooklyn|16.712644158255017|           8619|
|      Unknown|16.478047605140187|           6848|
|    Manhattan|11.456650322558236|         921229|
+-------------+------------------+---------------+



##### Taxis should target the Queens borough given the fact that the number of trips , tips and avg fare are higher on average than the other borough.
##### Staten Island and EWR showing high avg_fare however number of trips are very less.

#### 8.Tip and Fare effect analysis based on pickup location type

In [0]:
btf = spark.sql("(Select 'tourist' as pickup_location, avg(fare_amount) avg_fare, avg(tip_amount) avg_tip,count(*) from TD  where tourist_pu == 1) \
          union \
          (Select 'entertainment' as pickup_location, avg(fare_amount) avg_fare, avg(tip_amount) avg_tip,count(*) from TD  where entert_pu == 1) \
          union \
          (Select 'park' as pickup_location, avg(fare_amount) avg_fare, avg(tip_amount) avg_tip,count(*) from TD  where park_pu == 1) \
          union \
          (Select 'workplace' as pickup_location, avg(fare_amount) avg_fare, avg(tip_amount) avg_tip,count(*) from TD  where workplace_pu == 1) \
          union \
          (Select 'residential' as pickup_location, avg(fare_amount) avg_fare, avg(tip_amount) avg_tip,count(*) from TD  where residential_pu == 1)"                 
         )
btf.show()
display(btf)

+---------------+------------------+------------------+--------+
|pickup_location|          avg_fare|           avg_tip|count(1)|
+---------------+------------------+------------------+--------+
|        tourist|21.303031673310723| 4.662185465050958|  151263|
|  entertainment|11.406063612788005|2.7407718643154477|  653988|
|           park|11.194850103888395|2.6754489462748543|   40428|
|      workplace|14.396744463163085|3.2710847680486435|   24337|
|    residential| 16.83525601092896|3.0536172131147508|   36600|
+---------------+------------------+------------------+--------+



pickup_location,avg_fare,avg_tip,count(1)
tourist,21.303031673310723,4.662185465050958,151263
entertainment,11.406063612788005,2.7407718643154477,653988
park,11.194850103888395,2.6754489462748543,40428
workplace,14.396744463163085,3.2710847680486435,24337
residential,16.83525601092896,3.0536172131147508,36600


##### Taxis should primarily target touristic areas given the fact that the tips and avg fare are higher on average than the other pickup location